In [6]:
import requests
import pandas as pd

In [7]:
r = requests.get('https://api.beta.ons.gov.uk/v1/datasets')

In [8]:
type(r.json()['items'][2])

dict

In [9]:
len(r.json()['items'])

38

In [13]:
dataset_titles = []
dataset_descriptions = []
dataset_ids = []
for i in range(len(r.json()['items'])):
    dataset_titles.append(r.json()['items'][i]['title'])
    dataset_descriptions.append(r.json()['items'][i]['description'])
    dataset_ids.append(r.json()['items'][i]['id'])

table_of_codes = pd.DataFrame(data={'title': dataset_titles,
                   'description': dataset_descriptions,
                  'dataset_id': dataset_ids})

In [14]:
table_of_codes.head()

# TODO: build one chart from the above datasets

,title,description,dataset_id
0,Consumer Prices Index including owner occupier...,CPIH is the most comprehensive measure of infl...,cpih01
1,"Population Estimates for UK, England and Wales...",Estimates of the usual resident population for...,mid-year-pop-est
2,"Annual summary of hours worked, place of work ...",Estimates of paid hours worked for UK employee...,ashe-table-7-hours
3,"Annual summary of earnings, place of work by L...","Estimates of weekly, hourly and annual earning...",ashe-table-7-earnings
4,"Annual summary of hours worked, place of resid...",Estimates of paid hours worked for UK employee...,ashe-table-8-hours


## find information about a dataset
we need to know the following items to make a correct api call:
* edition
* version
* time
* aggregate 
* geography
* dimension

> This allows querying of a single observation/value by providing one option per dimension, but will also allow one of these dimensions to be a ‘wildcard’ and return all values for this dimension.

`/datasets/{datasetId}/editions/{edition}/versions/{version}/observations?time={timeLabel}&geography={geographyID}&dimension3={dimension3ID}&dimension4={dimension4ID}...`

In [20]:
def post(extension, is_ok=False):
    r = requests.get(f'https://api.beta.ons.gov.uk/v1/' + extension)
    if is_ok:
        print(r)
    return r.json()

`/datasets/cpih01/editions/time-series/versions/6/observations?time=Oct-11&geography=K02000001&aggregate=cpih1dim1A0`

### editions param

In [21]:
editions = post('/datasets/cpih01/editions')

In [22]:
editions['items'][0]['edition']

'time-series'

In [ ]:
my_dict = post('/datasets/cpih01/editions/time-series/versions')['items'][3]
mother_df = pd.DataFrame.from_dict(my_dict, orient='index').T

In [ ]:
for i in range(len(post('/datasets/cpih01/editions/time-series/versions')['items'])):
    try:
        my_dict = post('/datasets/cpih01/editions/time-series/versions')['items'][i+1]
        df = pd.DataFrame.from_dict(my_dict, orient='index').T
        mother_df = mother_df.append(df)
    except:
        pass


In [ ]:
mother_df

* find out how many versions there are
* for each version, grab all the dimensions
* select a trivial day in the time slot so we can see all the 

In [ ]:
post('/code-lists/cpih01/', is_ok=True)

In [ ]:

post('/datasets/cpih01/editions/time-series/versions/6')['dimensions']

`observations?time={timeLabel}&geography={geographyID}&dimension3={dimension3ID}&dimension4={dimension4ID}`

In [ ]:
post('/datasets/cpih01/editions/time-series/versions/30/observations?time=Oct-11&geography=K02000001&aggregate=*').keys()

In [ ]:
# post('/datasets/cpih01/editions/time-series/versions/30/observations?time=Oct-11&geography=K02000001&aggregate=*')\
#     ['observations'][0]['dimensions']['aggregate']['id']

In [ ]:
post('/datasets/cpih01/editions/time-series/versions/30/observations?time=Jan-03&geography=K02000001&aggregate=*')\
    ['observations'][0]['dimensions']['aggregate']['label']

In [ ]:
for dimension in post('/datasets/cpih01/editions/time-series/versions/30/observations?time=Jan-03&geography=K02000001&aggregate=*')\
        ['observations'][0]['dimensions'].keys():
    print(dimension)
    
    for i in range(20):
    post('/datasets/cpih01/editions/time-series/versions/30/observations?time=Jan-03&geography=K02000001&aggregate=*')\
        ['observations'][i]['dimensions']['aggregate']['label']

In [ ]:
# refactor this so we make one call, then iterate through the one call. rather than making unique calls for each loop
for i in range(1):
    post('/datasets/cpih01/editions/time-series/versions/30/observations?time=Jan-03&geography=K02000001&aggregate=*')\
    ['observations'][i]['dimensions']['aggregate']['label']

In [ ]:
post('/code-lists')['items']

In [ ]:
post('code-lists/wellbeing-estimate')